N.b. This requries a local large language model to be running. Its rquired that users download OLLAMA before they can do anything https://github.com/ollama/ollama 

The Ollama server also needs to be running for this code to work

I.e. #ollama run llama3.2

In [105]:
from itertools import combinations
from pyvis.network import Network
import networkx as nx
import numpy as np
import pandas as pd
import re
import json
import copy
import nltk
from nltk.tokenize import sent_tokenize
from google import genai
import ollama
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from nltk.tokenize import word_tokenize
nltk.download("punkt")

[nltk_data] Downloading package punkt to /Users/svztg/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [97]:
datasets = []
publications = []
tools = []
with open ('./data/datasets.json') as f:
    datasets = json.load(f)
with open ('./data/publications.json') as f:
    publications = json.load(f)
with open ('./data/tools.json') as f:
    tools = json.load(f)

len(datasets), len(publications), len(tools)

(1059, 3199, 243)

In [98]:
titles = [x['metadata']['summary']['title'] for x in datasets]
titles[:5]

['Cystic Fibrosis Patient Microbiology Cultures',
 'Cystic Fibrosis Patient Liver Enzyme',
 'Cystic Fibrosis Patient Annual Review Encounters',
 'Cystic Fibrosis Patient Transplants',
 'Cystic Fibrosis Patient Sweat Tests']

In [99]:
descriptions = [x['metadata']['summary']['description'] for x in datasets]
descriptions[:5]

["The UK CF Registry is a centralised database of all 60 CF centres across the UK. Data are manually entered in calendar years by CF clinical teams for the 99% of people with a diagnosis of CF who consent to their data being donated to the Registry. Data are entered onto a secure web-portal. For more information please see www.cysticfibrosis.org.uk/registry and 'Data Resource Profile: The UK CF Registry' published in the International Journal of Epidemiology (2018 Feb 1;47(1)9-10e).",
 "The UK CF Registry is a centralised database of all 60 CF centres across the UK. Data are manually entered in calendar years by CF clinical teams for the 99% of people with a diagnosis of CF who consent to their data being donated to the Registry. Data are entered onto a secure web-portal. For more information please see www.cysticfibrosis.org.uk/registry and 'Data Resource Profile: The UK CF Registry' published in the International Journal of Epidemiology (2018 Feb 1;47(1)9-10e).",
 "The UK CF Registry

In [112]:
paper_abstracts = [x['abstract'] for x in publications]
paper_doi = [x['paper_doi'] for x in publications]
paper_abstracts[1]
abstract = paper_abstracts[50]

['https://doi.org/10.1371/journal.pone.0220771', 'https://doi.org/10.1136/bmjopen-2018-026290', 'https://doi.org/10.1016/s0140-6736(20)30854-0', 'https://doi.org/10.1093/ageing/afz110', 'https://doi.org/10.1183/13993003.01809-2020', 'https://doi.org/10.1038/s41598-019-42036-w', 'https://doi.org/10.1136/jech-2017-210370', 'https://doi.org/10.12688/wellcomeopenres.15151.1', 'https://doi.org/10.2196/16452', 'https://doi.org/10.1038/s41598-019-48927-2', 'https://doi.org/10.1002/cnm.3255', 'https://doi.org/10.1136/jech-2020-214051', 'https://doi.org/10.1038/s42254-020-0178-4', 'https://doi.org/10.1016/j.molmet.2020.01.009', 'https://doi.org/10.1186/s12913-019-4286-8', 'https://doi.org/10.1016/j.eclinm.2020.100296', 'https://doi.org/10.3389/fgene.2019.00922', 'https://doi.org/10.1002/cnm.3180', 'https://doi.org/10.1136/bmjopen-2019-033424', 'https://doi.org/10.1161/jaha.119.012812', 'https://doi.org/10.1186/s12942-020-00208-2', 'https://doi.org/10.1002/bjs.11580', 'https://doi.org/10.1177/20

/var/folders/5t/sdyz102s5ls5vj828z703jv00000gp/T/ipykernel_81588/778616879.py:5: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython.display
  from IPython.core.display import HTML, display


In [101]:
def can_you_find_a_dataset(description):
    prompt = (
        "The text below is from a research publication, can you the names of any datasets referenced?"
        "In the response, please give me a comma separated list of names of possible datasets, if you dont think there is one, "
        "dont make up any and return 'none':\n\n" + description
    )
    
    response = ollama.chat(
        model="llama3.2:latest",
        messages=[{"role": "user", "content": prompt}],
    )
    return response

In [107]:

def extract_response(response_text: str) -> str:
     # Ensure that the response_text is a string.
    if not isinstance(response_text, str):
        response_text = str(response_text)
    
    # This regex captures any characters between "content='" and the next "'"
    pattern = r"content='(.*?)'"
    match = re.search(pattern, response_text)
    if match:
        return match.group(1)
    return ""

In [103]:
def check_output(description):
    prompt = (
        "does the following look like a comma seperated list of datasets? give a simple yes or no response" + description
    )
    
    response = ollama.chat(
        model="llama3.2:latest",
        messages=[{"role": "user", "content": prompt}],
    )
    return response


In [109]:
def get_dataset_names(abstract: str, max_attempts: int = 3) -> str:
    attempt = 0
    valid_datasets = set()

    # Split abstract into sentences
    sentences = sent_tokenize(abstract)

    while attempt < max_attempts:
        for sentence in sentences:
            # Extract dataset names from the sentence
            response = can_you_find_a_dataset(sentence)
            dataset_names = extract_response(response)
            #print(f"Attempt {attempt + 1}, Sentence: {sentence}")
            #print(f"Extracted dataset(s): {dataset_names}")

            # Check if extraction is valid
            check_resp = check_output(dataset_names)
            check_result = extract_response(check_resp)
            #print(f"Validation output: {check_result}")

            # Only keep valid dataset names, discard 'none' responses
            if check_result.strip().lower() == "yes":
                valid_datasets.update(dataset_names.split(", "))

        if valid_datasets:
            return ", ".join(valid_datasets)

        attempt += 1

    return ""

dataset_names = get_dataset_names(abstract)
print("names=",dataset_names)

Attempt 1, Sentence: 'METHODS Study design First, we will conduct an open prospective cohort study to determine uptake of vaccine and compare the risk of outcomes (COVID-19 infection, hospitalisation, ICU admissions, and death) amongst first dose vaccinated and second dose vaccinated to people who have not yet been vaccinated.
Extracted dataset(s): none
Validation output: No
Attempt 1, Sentence: In addition, a Test Negative Design (TND) case control study for infection nested within this national cohort will compare outcomes of interest between COVID-19 vaccine doses in CYP and unvaccinated controls.
Extracted dataset(s): 
Validation output: Yes.
Attempt 1, Sentence: Population and setting Children and young people aged 0-17 years who are resident in Scotland.
Extracted dataset(s): none
Validation output: No
Attempt 1, Sentence: We will exclude the following: persons aged 18 or more at the study start date; deceased prior to the study start date.
Extracted dataset(s): None
Validation o

KeyboardInterrupt: 